In [17]:
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import tkinter as Tk
import matplotlib.animation as animation
import peakutils

In [18]:
import pyaudio
import numpy as np
import matplotlib.pyplot as plt
from tkinter import TclError
import struct
from scipy.io.wavfile import write
import wave

In [19]:
FORMAT = pyaudio.paInt16 # 16bit format per sample
RATE = 44100 # Samples per second
CHANNELS = 1 # Only one source , your microphone
CHUNK = 1024 # We split our data into 1024 samples each time , for utilizing processing power

In [20]:

class Record:
    def __init__(self,Format,rate,chunk,channels):
            # Initiate details
            self.chunk = chunk
            self.format = Format
            self.channels = channels
            self.rate = rate

            self.audio = pyaudio.PyAudio()
            self.stream = self.audio.open(format = self.format,
                                          channels = self.channels,
                                          rate = self.rate, 
                                          input=True)
                                          #output=True,
                                          #frames_per_buffer=self.chunk)
            
    def make_plots(self):
        self.fig = plt.Figure(figsize=(20, 10))
        self.af = np.linspace(0, self.rate, self.chunk//2 +1)
        root = Tk.Tk()
        self.label = Tk.Label(root,text="Παναγιωτης Σακκουλης").grid(column=0, row=0)
        self.canvas = FigureCanvasTkAgg(self.fig, master=root)
        self.canvas.get_tk_widget().grid(column=0,row=1)
        self.ax = self.fig.add_subplot()
        self.line, = self.ax.plot(self.af, np.random.rand(self.chunk//2 +1))
        self.ax.set_xlabel('Frequency (Hz)')
        self.ax.set_ylabel('Power (dB)')
        self.ax.set_title('Fourier')
        xps=0
        yps=0
        text= "x={:.3f}, y={:.3f}".format(xps, yps) # Text at the pointer of max
        self.an=self.ax.annotate(text,
        xy=(xps, yps), xycoords='data',
        xytext=(-15, 25), textcoords='offset points',
        arrowprops=dict(facecolor='black', shrink=0.05),
        horizontalalignment='left', verticalalignment='bottom') # Create the array

In [ ]:
if __name__ == '__main__':
    sound=Record(FORMAT,RATE,CHUNK,CHANNELS)
    sound.make_plots()
    np.seterr(divide = 'ignore')
    frames=[]

    def animate(i):
        sound.an.remove()
        if(i%(sound.rate//sound.chunk)==0)and(i>0):
            print(int(i/(sound.rate//sound.chunk)),'Seconds Recorded')
        in_data=sound.stream.read(sound.chunk)
        frames.append(in_data)
        audio_data = np.frombuffer(in_data, np.int16)
        dfft=10*np.log10(abs(np.fft.rfft(audio_data))) #  Fast Fourier Transform, 10*log10(abs) is to scale it to dB
        xf = np.fft.rfftfreq(audio_data.size, d=1.0/sound.rate)
        
        data_int = struct.unpack(str(2 * sound.chunk) + 'B', in_data)  # Turn int into bit
        data_np = np.array(data_int, dtype='b')[::2] +128
        
        xmid = np.argmax(dfft) # the index of the max price
        x=xf[xmid]
        
        y = np.amax(dfft) # The actual max
        sound.ax.set_ylim(max(0,min(dfft))-10, min(20000,max(dfft))+10)
        sound.ax.set_xlim(0, 20000)
        sound.line.set_data(xf,dfft)  # update the data
        text= "x={:.3f}, y={:.3f}".format(x, y) # Text at the pointer of max
        sound.an=sound.ax.annotate(text,
        xy=(x, y), xycoords='data',
        xytext=(-15, 25), textcoords='offset points',
        arrowprops=dict(facecolor='black', shrink=0.05),
        horizontalalignment='left', verticalalignment='bottom') # Create the array
        return sound.line,
    
    
    ani = animation.FuncAnimation(sound.fig, animate, interval=1, blit=False)
    Tk.mainloop()
    print("DONE")
    sound.stream.stop_stream() # Stop the stream
    sound.stream.close()
    
    wf = wave.open("fft.wav", 'wb')
    wf.setnchannels(sound.channels)
    wf.setsampwidth(sound.audio.get_sample_size(sound.format))
    wf.setframerate(sound.rate)
    wf.writeframes(b''.join(frames)) # we save it as byte type 
    wf.close()     
       

1 Seconds Recorded
2 Seconds Recorded
